In [ ]:
#default_exp data.pipeline

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.notebook.showdoc import show_doc

In [ ]:
torch.cuda.set_device(6)

# Transforms and Pipeline

> Low-level transform pipelines

The classes here provide functionality for creating *partially reversible functions*, which we call `Transform`s. By "partially reversible" we mean that a transform can be `decode`d, creating a form suitable for display. This is not necessarily identical to the original form (e.g. a transform that changes a byte tensor to a float tensor does not recreate a byte tensor when decoded, since that may lose precision, and a float tensor can be displayed already.)

Classes are also provided and for composing transforms, and mapping them over collections. The following functionality is provided:

- A `Transform` is created with an `encodes` and potentially `decodes` function. 
- `Pipeline` is a transform which composes transforms
- `TfmdList` takes a collection and a transform, and provides an indexer (`__getitem__`) which dynamically applies the transform to the collection items.

## Convenience functions

In [ ]:
def get_func(t, name, *args, **kwargs):
    "Get the `t.name` (potentially partial-ized with `args` and `kwargs`) or `noop` if not defined"
    f = getattr(t, name, noop)
    return f if not (args or kwargs) else partial(f, *args, **kwargs)

This works for any kind of `t` supporting `getattr`, so a class or a module.

In [ ]:
test_eq(get_func(operator, 'neg', 2)(), -2)
test_eq(get_func(operator.neg, '__call__')(2), -2)
test_eq(get_func(list, 'foobar')([2]), [2])
t = get_func(torch, 'zeros', dtype=torch.int64)(5)
test_eq(t.dtype, torch.int64)
a = [2,1]
get_func(list, 'sort')(a)
test_eq(a, [1,2])

In [ ]:
#export
def show_title(o, ax=None, ctx=None):
    "Set title of `ax` to `o`, or print `o` if `ax` is `None`"
    ax = ifnone(ax,ctx)
    if ax is None: print(o)
    else: ax.set_title(o)

In [ ]:
test_stdout(lambda: show_title("title"), "title")

## Func -

Tranforms, data augmentation in particular, are built in fastai around patching some types. For instance a `TensorImage` object isn't flipped the same way as a `TensorPoints` or a `TensorBBox` object, so we will want to access the `flip` attribute of each of those classes. The following class allows us to create a generic object associated to that `flip` name that will then look for this attribute in the type (or list of types) we feed it.

In [ ]:
class Func():
    "Basic wrapper around a `name` with `args` and `kwargs` to call on a given type"
    def __init__(self, name, *args, **kwargs): self.name,self.args,self.kwargs = name,args,kwargs
    def __repr__(self): return f'sig: {self.name}({self.args}, {self.kwargs})'
    def _get(self, t): return get_func(t, self.name, *self.args, **self.kwargs)
    def __call__(self,t): return L(t).mapped(self._get) if is_listy(t) else self._get(t)

You can call the `Func` object on any module name or type, even a list of types. It will return the corresponding function (with a default to `noop` if nothing is found) or list of functions.

In [ ]:
test_eq(Func('sqrt')(math), math.sqrt)
test_eq(Func('sqrt')(torch), torch.sqrt)

@patch
def powx(x:math, a): return math.pow(x,a)
@patch
def powx(x:torch, a): return torch.pow(x,a)
tst = Func('powx',a=2)([math, torch])
test_eq([f.func for f in tst], [math.powx, torch.powx])
for t in tst: test_eq(t.keywords, {'a': 2})

In [ ]:
class _Sig():
    def __getattr__(self,k):
        def _inner(*args, **kwargs): return Func(k, *args, **kwargs)
        return _inner

Sig = _Sig()

In [ ]:
show_doc(Sig, name="Sig")

<h4 id="<code>Sig</code>" class="doc_header"><code>Sig</code><a href="https://github.com/fastai/fastai_docs/tree/master/dev/__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>Sig</code>(**\*`args`**, **\*\*`kwargs`**)



`Sig` is just sugar-syntax to create a `Func` object more easily with the syntax `Sig.name(*args, **kwargs)`.

In [ ]:
f = Sig.sqrt()
test_eq(f(math), math.sqrt)
test_eq(f(torch), torch.sqrt)

In [ ]:
class SelfFunc():
    "Search for `name` attribute and call it with `args` and `kwargs` on any object it's passed."
    def __init__(self, nm, *args, **kwargs): self.nm,self.args,self.kwargs = nm,args,kwargs
    def __repr__(self): return f'self: {self.nm}({self.args}, {self.kwargs})'
    def __call__(self, o):
        if not is_listy(o): return getattr(o,self.nm)(*self.args, **self.kwargs)
        else: return [getattr(o_,self.nm)(*self.args, **self.kwargs) for o_ in o]

The difference between `Func` and `SelfFunc` is that `Func` will generate a function when you call it on a type. On the other hand, `SelfFunc` is already a function and each time you call it on an object it looks for the `name` attribute and call it on `args` and `kwargs`.

In [ ]:
tst = SelfFunc('sqrt')
x = torch.tensor([4.])
test_eq(tst(x), torch.tensor([2.]))
assert isinstance(tst(x), Tensor)

In [ ]:
class _SelfFunc():
    def __getattr__(self,k):
        def _inner(*args, **kwargs): return SelfFunc(k, *args, **kwargs)
        return _inner
    
Self = _SelfFunc()

In [ ]:
show_doc(Self, name="Self")

<h4 id="<code>Self</code>" class="doc_header"><code>Self</code><a href="https://github.com/fastai/fastai_docs/tree/master/dev/__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>Self</code>(**\*`args`**, **\*\*`kwargs`**)



`Self` is just syntax sugar to create a `SelfFunc` object more easily with the syntax `Self.name(*args, **kwargs)`.

In [ ]:
f = Self.sqrt()
x = torch.tensor([4.])
test_eq(f(x), torch.tensor([2.]))
assert isinstance(f(x), Tensor)

## Transform -

In [ ]:
def _filter_with_type(f, t):
    if is_listy(f): return f
    sig = inspect.signature(f)
    t_in = [p.annotation if p.annotation != inspect._empty else None 
            for p in sig.parameters.values() if p.default == inspect._empty and p.kind != inspect._VAR_KEYWORD]
    if len(t_in) > 1: return f
    return [f if t_ is None or t_in[0] is None or t_==t_in[0] else noop for t_ in t]

In [ ]:
class Transform(PrePostInit):
    "A function that `encodes` if `filt` matches, and optionally `decodes`"
    order,filt = 0,None
    def __init__(self,encodes=None,decodes=None):
        if encodes is not None: self.encodes = encodes
        if decodes is not None: self.decodes = decodes
    
    def _filt_match(self, filt): return self.filt is None or self.filt==filt
    def _apply(self, fs, x, filt):
        if not self._filt_match(filt): return x
        if is_listy(fs): return tuple(f(x_) for f,x_ in zip(fs,x))
        return fs(*L(x))
    
    def __call__(self, x, filt=None): return self._apply(self.encodes, x, filt)
    def decode  (self, x, filt=None): return self._apply(self.decodes, x, filt)
    def __getitem__(self, x): return self(x) # So it can be used as a `Dataset`

    def _filter_with_type(self, t):
        if self.encodes is None: self.encodes = noop
        if self.decodes is None: self.encodes = noop
        if is_listy(t): self.encodes = _filter_with_type(self.encodes, t)
        if is_listy(t): self.decodes = _filter_with_type(self.decodes, t)
            
add_docs(Transform,
         __call__="Call `self.encodes` unless `filt` is passed and it doesn't match `self.filt`",
         decode  ="Call `self.decodes` unless `filt` is passed and it doesn't match `self.filt`")

In a transformation pipeline some steps need to be reversible - for instance, if you turn a string (such as *dog*) into an int (such as *1*) for modeling, then for display purposes you'll want to turn it back to a string again (e.g. when you have a prediction). In addition, you may wish to only run the transformation for a particular data subset, such as the training set.

`Transform` provides all this functionality. `filt` is some dataset index (e.g. provided by `DataSource`), and you provide `encodes` and optional `decodes` functions for your code. You can pass `encodes` and `decodes` functions directly to the constructor for quickly creating simple transforms.

In [ ]:
tfm = Transform(operator.neg, decodes=operator.neg)
start = 4
t = tfm(start)
test_eq(t, -4)
test_eq(t, tfm[start]) #You can use a transform as a dataset
test_eq(tfm.decode(t), start)

In [92]:
class _AddOne(Transform):
    filt=1
    def encodes(self, x): return x+1
    def decodes(self, x): return x-1

addt = _AddOne()
test_eq(addt(start,filt=1), 5)
test_eq(addt(start,filt=0), start)

At some point in the data-collection pipeline, your objects will be tuples (usually input,label). There are then different behaviors you might want your `Transform` to adopt such as:
- being applied to the tuple and returning a new tuple
- being applied to each part of the tuple
- being applied to some parts of the tuple but not all

You can control which behavior will be used with the signature of your `encodes` function. If it accepts several arguments (without defaults), then the transform will be applied on the tuple and expected to return a tuple. If your `encodes` function only accepts one argument, it will be applied on every part of the tuple. You can even control which part of the tuples with a type annotation: the tranform will only be applied to the items in the tuple that correspond to that type.

All of this is enabled the private method `_filter_with_type` that is called in the setup of a `Pipeline` (so out of the blue your transform object won't have this behavior). The `Pipeline` will analyze the type of objects (as given by the return annotation of any transform) and pass them along, wich tells the transform it will receive a given type (or a tuple of given types).

In [ ]:
#Apply on the tuple as a whole
class _Add(Transform):
    def encodes(self, x, y): return (x+y,y)
    def decodes(self, x, y): return (x-y,y)

addt = _Add()
addt._filter_with_type([float,float])
t = addt([1,2])
test_eq(t, (3,2))
test_eq(addt.decode(t), (1,2))

In [ ]:
#Apply on all part of the tuple
class _AddOne(Transform):
    def encodes(self, x): return x+1
    def decodes(self, x): return x-1

addt = _AddOne()
addt._filter_with_type([float,float])
t = addt([1,2])
test_eq(t, (2,3))
test_eq(addt.decode(t), (1,2))

In [88]:
#Apply on all float of the tuple
#Also note that your tuples can have more than two elements
class _AddOne(Transform):
    def encodes(self, x:float): return x+1
    def decodes(self, x:float): return x-1

addt = _AddOne()
addt._filter_with_type([int,float, float])
t = addt([1,2,3])
test_eq(t, (1,3,4))
test_eq(addt.decode(t), (1,2,3))

In [94]:
def mk_func(f, t):
    "Make `f` a function with type `t`"
    if isinstance(f,str ): f = Func(f)
    if isinstance(f,Func): f = f(t)
    return f

In [95]:
def mk_tfm(f,t):
    "Make `f` a transform with type `t`"
    if not is_listy(f): f = (f,None)
    return Transform(mk_func(f[0],t), mk_func(f[1],t))

## Pipeline -

In [96]:
def compose_tfms(x, tfms, func_nm='encode', reverse=False):
    if reverse: tfms = reversed(tfms)
    for tfm in tfms: x = getattr(tfm,func_nm,noop)(x)
    return x

In [97]:
#export
def _get_ret(func):
    ann = getattr(func,'__annotations__', None)
    if not ann: return None
    return ann.get('return')

In [98]:
#export
class Pipeline():
    def __init__(self, funcs):
        self.raw_fs = funcs
        
    def setup(self, t=None):
        self.fs,self.t_show = [],None
        if len(self.raw_fs) == 0: self.final_t = t
        else:
            for i,f in enumerate(self.raw_fs):
                if not isinstance(f,Transform): f = mk_tfm(f, t)
                f._filter_with_type(t)
                self.fs.append(f)
                if hasattr(t, 'show') and self.t_show is None:
                    self.t_idx,self.t_show = i,t
                t = _get_ret(f.encodes) or t
            if hasattr(t, 'show') and self.t_show is None:
                self.t_idx,self.t_show = i+1,t
            self.final_t = t
                
    def __call__(self, o): return compose_tfms(o, self.fs)
    def decode  (self, i): return compose_tfms(i, self.fs, func_nm='decode', reverse=True)
    
    def show(self, o, ctx=None, **kwargs):
        if self.t_show is None: return self.decode(o)
        o = compose_tfms(o, self.fs[self.t_idx:], func_nm='decode', reverse=True)
        return self.t_show.show(o, ctx=ctx, **kwargs)
    

    #def __call__(self, x, **kwargs): return self.composed(x, **kwargs)
    #def __getitem__(self, x): return self(x)
    #def decode_at(self, idx): return self.decode(self[idx])
    #def show_at(self, idx): return self.show(self[idx])
    #def __repr__(self): return str(self.tfms)
    

In [ ]:
add_docs(Pipeline,
         "A pipeline of composed (for encode/decode) transforms, setup one at a time",
         __call__="Compose `__call__` of all `tfms` on `x`",
         decode="Compose `decode` of all `tfms` on `x`",
         decode_at="Decoded item at `idx`",
         show_at="Show item at `idx`",
         show="Show item",
         delete="Delete transform `idx` from pipeline",
         remove="Remove `tfm` from pipeline",
         set_tupled="Set any `MappedTransform`s in `tfms` to tupled mode")

A list of transforms are often applied in a particular order, and decoded by applying in the reverse order. `Pipeline` provides this functionality, and also ensures that any `setup` methods are called, without including later transforms in those calls. NB: `setup` must be run before encoding/decoding.

Here's some simple examples:

In [ ]:
# Empty pipelines are a noop
pipe = Pipeline()
pipe.setup()
test_eq(pipe(1), 1)

In [ ]:
# Check a standard pipeline
pipe = Pipeline([negtfm(),floattfm()])
pipe.setup()

start = 2
t = pipe(2)
test_eq(t, -2.0)
test_eq(type(t), float)
test_eq(t, pipe[2])
test_eq(pipe.decode(t), start)
# `show` is on `tfloat` so `show_at` decodes that tfm only
test_stdout(lambda:pipe.show_at(1), '-1')
test_eq(pipe.assoc, Item)

In [ ]:
# Check opposite order
pipe = Pipeline([floattfm(),negtfm()])
pipe.setup()

t = pipe(2)
test_eq(t, -2.0)
# `show` is on `tfloat` so needs to decode negtfm first
test_stdout(lambda:pipe.show_at(1), '1')
test_eq(pipe.assoc, Item)

### Methods

In [ ]:
show_doc(Pipeline.__call__)

<h4 id="<code>Pipeline.__call__</code>" class="doc_header"><code>Pipeline.__call__</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#Pipeline--" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.__call__</code>(**`x`**, **\*\*`kwargs`**)

Compose `__call__` of all `tfms` on `x`

In [ ]:
show_doc(Pipeline.decode)

<h4 id="<code>Pipeline.decode</code>" class="doc_header"><code>Pipeline.decode</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#Pipeline--" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.decode</code>(**`x`**, **\*\*`kwargs`**)

Compose `decode` of all `tfms` on `x`

In [ ]:
show_doc(Pipeline.delete)

<h4 id="<code>Pipeline.delete</code>" class="doc_header"><code>Pipeline.delete</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#Pipeline--" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.delete</code>(**`idx`**)

Delete transform `idx` from pipeline

In [ ]:
show_doc(Pipeline.remove)

<h4 id="<code>Pipeline.remove</code>" class="doc_header"><code>Pipeline.remove</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#Pipeline--" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.remove</code>(**`tfm`**)

Remove `tfm` from pipeline

In [ ]:
show_doc(Pipeline.add)

<h4 id="<code>Pipeline.add</code>" class="doc_header"><code>Pipeline.add</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#Pipeline--" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.add</code>(**`tfms`**, **`items`**=*`None`*)

Call `setup` on all `tfms` and append them to this pipeline

In [ ]:
show_doc(Pipeline.show_at)

<h4 id="<code>Pipeline.show_at</code>" class="doc_header"><code>Pipeline.show_at</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#Pipeline--" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.show_at</code>(**`idx`**)

Show item at `idx`

In [ ]:
show_doc(Pipeline.decode_at)

<h4 id="<code>Pipeline.decode_at</code>" class="doc_header"><code>Pipeline.decode_at</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#Pipeline--" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.decode_at</code>(**`idx`**)

Decoded item at `idx`

In [ ]:
#export
def make_tfm(tfm):
    "Create a `Pipeline` (if `tfm` is listy) or a `Transform` otherwise"
    if isinstance(tfm,Pipeline): return tfm
    return Pipeline(tfm) if is_listy(tfm) else Transform.create(tfm)

## TfmdList -

In [ ]:
#export
@docs
class TfmdList(GetAttr):
    "A transform applied to a collection of `items`"
    _xtra = 'decode __call__ show assoc'.split()
    
    def __init__(self, items, tfm, do_setup=True):
        self.items = L(items)
        self.default = self.tfm = make_tfm(tfm)
        if do_setup: self.setup()

    def __getitem__(self, i):
        "Transformed item(s) at `i`"
        its = self.items[i]
        return its.mapped(self.tfm) if is_iter(i) else self.tfm(its)

    def decode_batch(self, b, **kwargs):
        "Decode `b`, a list of lists of pipeline outputs (i.e. output of a `DataLoader`)"
        transp = L(zip(*L(b)))
        return transp.mapped(self.decode, **kwargs).zipped()

    def setup(self): getattr(self.tfm,'setup',noop)(self)
    def subset(self, idxs): return self.__class__(self.items[idxs], self.tfm, do_setup=False)
    def decode_at(self, idx): return self.decode(self[idx])
    def show_at(self, idx): return self.show(self[idx])
    def __eq__(self, b): return all_equal(self, b)
    def __len__(self): return len(self.items)
    def __iter__(self): return (self[i] for i in range_of(self))
    def __repr__(self): return f"{self.__class__.__name__}: {self.items}\ntfms - {self.tfm}"
    
    _docs = dict(setup="Transform setup with self",
                 decode_at="Decoded item at `idx`",
                 show_at="Show item at `idx`",
                 subset="New `TfmdList` that only includes items at `idxs`")

In [ ]:
pipe = Pipeline([negtfm(),floattfm()])
pipe.setup()

tl = TfmdList([1,2,3], pipe)
t = tl[1]
test_eq(t, -2.0)
test_eq(type(t), float)
test_eq(tl.decode_at(1), 2)
test_eq(tl.decode(t), 2)
test_stdout(lambda: tl.show_at(2), '-3')
tl

TfmdList: (#3) [1,2,3]
tfms - [<class 'local.core.negtfm'>, <class 'local.core.floattfm'>]

In [ ]:
p2 = tl.subset([0,2])
test_eq(p2, [-1.,-3.])

Here's how we can use `TfmdList.setup` to implement a simple category list, getting labels from a mock file list:

In [ ]:
class _Cat(Transform):
    assoc,order=Item,1
    def encodes(self, o): return self.o2i[o] if self._done_setup else o
    def decodes(self, o): return self.vocab[o]
    def setups(self, items): self.vocab,self.o2i = uniqueify(items, sort=True, bidir=True)

def _lbl(o): return o.split('_')[0]

test_fns = ['dog_0.jpg','cat_0.jpg','cat_2.jpg','cat_1.jpg','dog_1.jpg']
tcat = _Cat()
tl = TfmdList(test_fns, [tcat,_lbl])

test_eq(tcat.vocab, ['cat','dog'])
test_eq([1,0,0,0,1], tl)
test_eq(1, tl[-1])
test_eq([1,0], tl[0,1])
t = list(tl)
test_eq([1,0,0,0,1], t)
test_eq(['dog','cat','cat','cat','dog'], map(tl.decode,t))
test_stdout(lambda:tl.show_at(0), "dog")
tl

TfmdList: (#5) [dog_0.jpg,cat_0.jpg,cat_2.jpg,cat_1.jpg,dog_1.jpg]
tfms - [<function _lbl at 0x7f7cfab74158>, <class '__main__._Cat'>]

### Methods

In [ ]:
show_doc(TfmdList.__getitem__)

<h4 id="<code>TfmdList.__getitem__</code>" class="doc_header"><code>TfmdList.__getitem__</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmdList--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdList.__getitem__</code>(**`i`**)

Transformed item(s) at `i`

In [ ]:
tl.decode(tl[1])

'cat'

In [ ]:
test_eq(tl.decode_at(1),'cat')

In [ ]:
show_doc(TfmdList.show_at)

<h4 id="<code>TfmdList.show_at</code>" class="doc_header"><code>TfmdList.show_at</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmdList--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdList.show_at</code>(**`idx`**)

Show item at `idx`

In [ ]:
tl.show_at(1)

cat


## TfmOver -

In [ ]:
#export
class TfmOver(Transform):
    "Create tuple containing each of `tfms` applied to each of `o`"
    def __init__(self, tfms=None):
        if tfms is None: tfms = [None]
        self.activ,self.tfms = None,L(tfms).mapped(Pipeline)

    def __call__(self, o, *args, **kwargs):
        "List of output of each of `tfms` on `o`"
        if self.activ is not None: return self.tfms[self.activ](o[self.activ], *args, **kwargs)
        return [t(p, *args, **kwargs) for p,t in zip(o,self.tfms)]
    
    def show(self, o, ctx=None, **kwargs):
        "Show result of `show` from each of `tfms`"
        for p,t in zip(o,self.tfms): ctx = t.show(p, ctx=ctx, **kwargs)
        return ctx

    def decode(self, o, **kwargs): return [t.decode(p, **kwargs) for p,t in zip(o,self.tfms)]
    def __repr__(self): return f'TfmOver({self.tfms})'

    def setups(self, o=None):
        "Setup each of `tfms` independently"
        for i,tfm in enumerate(self.tfms):
            self.activ = i
            tfm.setup(o)
        self.activ=None
    
    @property
    def assoc(self): return self.tfms.attrgot('assoc')
    
    @classmethod
    def piped(cls, tfms=None, final_tfms=None):
        "`Pipeline` that duplicates input, then maps `TfmOver` over `tfms`, optionally followed by any `final_tfms`"
        tfms = L(ifnone(tfms,[None]))
        init_tfm = partial(replicate,match=tfms)
        return Pipeline([init_tfm,cls(tfms)] + _set_tupled(final_tfms))

    xt,yt = add_props(lambda i,x:x.tfms[i])

In [ ]:
class _TNorm(Transform):
    assoc=Item
    def __init__(self): self.m,self.s = 0,1
    def encodes(self, o): return (o-self.m)/self.s
    def decodes(self, o): return (o*self.s)+self.m
    def setup(self, items):
        its = tensor(items)
        self.m,self.s = its.mean(),its.std()

In [ ]:
items = [1,2,3,4]
tl = TfmdList(items, TfmOver.piped([negtfm(), [negtfm(),_TNorm()]]))
x,y = zip(*tl)
test_close(tensor(y).mean(), 0)
test_close(tensor(y).std(), 1)
test_eq(x, [-1,-2,-3,-4])
test_stdout(lambda:tl.show_at(1), 'tensor(-2.)')
test_eq(tl.tfm.assoc, [None,Item])

In [ ]:
# Create a "batch"
b = list(zip(*tl))
bd = tl.decode_batch(b)

test_eq(len(bd),2)
test_eq(bd[0],items)
test_eq(bd[1],items)
test_eq(type(bd[1][0]),Tensor)
print('b ',b)
print('bd',bd)

b  [(-1, -2, -3, -4), (tensor(1.1619), tensor(0.3873), tensor(-0.3873), tensor(-1.1619))]
bd (#2) [(#4) [1,2,3,4],(#4) [tensor(1.),tensor(2.),tensor(3.),tensor(4.)]]


In [ ]:
# Empty tuplify
tp = TfmOver()
tp.setup()
test_eq(tp([1]), [1])

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 02_data_pipeline.ipynb.
Converted 03_data_external.ipynb.
Converted 04_data_core.ipynb.
Converted 05_data_source.ipynb.
Converted 06_vision_core.ipynb.
Converted 07_pets_tutorial-oo.ipynb.
Converted 07_pets_tutorial-oo1.ipynb.
Converted 07_pets_tutorial-oo2-meta.ipynb.
Converted 07_pets_tutorial.ipynb.
Converted 08_augmentation.ipynb.
Converted 10_layers.ipynb.
Converted 11_optimizer.ipynb.
Converted 12_learner.ipynb.
Converted 13_callback_schedule.ipynb.
Converted 14_callback_hook.ipynb.
Converted 15_callback_progress.ipynb.
Converted 16_callback_tracker.ipynb.
Converted 17_callback_fp16.ipynb.
Converted 90_notebook_core.ipynb.
Converted 91_notebook_export.ipynb.
Converted 92_notebook_showdoc.ipynb.
Converted 93_notebook_export2html.ipynb.
Converted 94_index.ipynb.
Converted 95_synth_learner.ipynb.
Converted tmp_tensor_inherit.ipynb.
